In [ ]:
%load_ext autoreload
%autoreload 2
import pyceres
import pycolmap
import numpy as np
from hloc.utils import viz_3d
from copy import deepcopy
import cv2

## Setup the toy example

In [ ]:
def sample_rotation(max_=np.pi * 2):
    aa = np.random.randn(3)
    aa *= np.random.rand() * max_ / np.linalg.norm(aa)
    return pycolmap.Rotation3d(aa)


def error(i_from_w, j_from_w):
    i_from_j = i_from_w * j_from_w.inverse()
    return np.rad2deg(i_from_j.rotation.angle()), np.linalg.norm(i_from_j.translation)


num = 20
i_from_w = [
    pycolmap.Rigid3d(sample_rotation(), np.random.rand(3) * 10) for _ in range(num)
]
i_from_j = [i_from_w[i] * i_from_w[(i + 1) % num].inverse() for i in range(num)]

i_from_w_init = [
    i_from_w[i] * pycolmap.Rigid3d(sample_rotation(np.pi / 5), np.random.rand(3))
    for i in range(num)
]
i_from_w_init[0] = i_from_w[0]

## PGO with relative poses

In [ ]:
i_from_w_opt = deepcopy(i_from_w_init)
err_opt = np.array([error(i_from_w[i], i_from_w_opt[i]) for i in range(num)])
print("initial:", np.mean(err_opt, 0))

prob = pyceres.Problem()
loss = pyceres.TrivialLoss()
costs = []
for i in range(num):
    cost = pycolmap.cost_functions.MetricRelativePoseErrorCost(i_from_j[i], np.eye(6))
    costs.append(cost)
    j = (i + 1) % num
    params = [
        i_from_w_opt[i].rotation.quat,
        i_from_w_opt[i].translation,
        i_from_w_opt[j].rotation.quat,
        i_from_w_opt[j].translation,
    ]
    prob.add_residual_block(cost, loss, params)
    prob.set_manifold(i_from_w_opt[i].rotation.quat, pyceres.EigenQuaternionManifold())
prob.set_parameter_block_constant(i_from_w_opt[0].rotation.quat)
prob.set_parameter_block_constant(i_from_w_opt[0].translation)

options = pyceres.SolverOptions()
options.linear_solver_type = pyceres.LinearSolverType.SPARSE_NORMAL_CHOLESKY
options.minimizer_progress_to_stdout = False
options.num_threads = -1
summary = pyceres.SolverSummary()
pyceres.solve(options, prob, summary)
print(summary.BriefReport())

err_opt = np.array([error(i_from_w[i], i_from_w_opt[i]) for i in range(num)])
print("optimized:", np.mean(err_opt, 0))

# PGO with absolute pose

In [ ]:
i_from_w_opt = deepcopy(i_from_w_init)

err_opt = np.array([error(i_from_w[i], i_from_w_opt[i]) for i in range(num)])
print("initial:", np.mean(err_opt, 0))

prob = pyceres.Problem()
loss = pyceres.TrivialLoss()
costs = []
for i in range(num):
    cost = pycolmap.cost_functions.AbsolutePoseErrorCost(i_from_w[i], np.eye(6))
    costs.append(cost)
    pose = i_from_w_opt[i]
    prob.add_residual_block(cost, loss, [pose.rotation.quat, pose.translation])
    prob.set_manifold(pose.rotation.quat, pyceres.EigenQuaternionManifold())

options = pyceres.SolverOptions()
options.linear_solver_type = pyceres.LinearSolverType.DENSE_QR
options.minimizer_progress_to_stdout = False
options.num_threads = -1
summary = pyceres.SolverSummary()
pyceres.solve(options, prob, summary)
print(summary.BriefReport())

err_opt = np.array([error(i_from_w[i], i_from_w_opt[i]) for i in range(num)])
print("optimized:", np.mean(err_opt, 0))